In [3]:

# Import necessary libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret
from functions import get_tracks_positions_in_playlists, fetch_playlist_tracks, fetch_and_display_playlist_info
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)

# Initialise the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

import logging
# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


In [6]:
# New Music Friday AU & NZ playlist 
playlist_id = '37i9dQZF1DWT2SPAYawYcO'

**TL;DR:** Extract all songs in the New Music Friday AU & NZ playlist. Then using all those unique track_id's, find which other playlists (from `playlists.json`) contain that track_id and what position it is in.

**Fetches and Displays Information About Tracks in a Specific Playlist and Their Positions in Various Playlists**

This function executes multiple steps to collect and display data:

1. **Fetch Playlist Tracks:** Utilizes `fetch_playlist_tracks` to retrieve track IDs, artist names, track names, and followers count for a given playlist ID. This involves Spotify API communication to return relevant data.

2. **Get Track Positions in Playlists:** With `get_tracks_positions_in_playlists`, it fetches tracks' positions across playlists defined in `playlists_dict`. This step queries Spotify API for each track's presence and position in these playlists.

3. **Prepare Data for DataFrame Creation:**
    - Prepares data by creating a list of dictionaries, each representing a track with its artist name, track title, and positions in playlists. Tracks not appearing in some playlists receive empty strings as values.
    - For each track in `track_positions`, initializes a new row with empty strings for each playlist, adds the artist's name and track title, and includes the track's position(s) in playlists. Positions are adjusted for zero-based indexing (`pos + 1`) and combined into a single comma-separated string.

4. **Convert List of Dictionaries to DataFrame:** Converts the prepared data into a pandas DataFrame, structured with 'Artist' and 'Track' followed by each playlist column. This DataFrame is designed for tabular display of each track's position across the specified playlists.

**Parameters:**

- `sp`: Spotify client instance for Spotify API communication.
- `playlists_dict`: Maps playlist names to Spotify IDs, identifying playlists for position checks.
- `playlist_id`: Spotify ID for the playlist being queried.

**Returns:**

- `df`: pandas DataFrame with track information and positions across playlists.


In [7]:
df = fetch_and_display_playlist_info(sp, playlists_dict, playlist_id)
df.head()

2024-02-17 21:42:52,692 - INFO - Fetching playlist: 37i9dQZF1DWT2SPAYawYcO
2024-02-17 21:42:53,922 - INFO - Fetched 84 items, increasing offset to 84.


,Artist,Track,Top 50 Australia,New Music Friday AU & NZ,Hot Hits Australia,Front Left,A1,Dance Generation,Get Popped!,R&B Connect,...,Breaking Hits,Chilled Hits,Vibe Check,crush,the hybrid,Just Chill,Morning Motivation,Lazy Sunday,Today's Top Hits,Singled Out
0,Beyoncé,TEXAS HOLD 'EM,,1,10,,,,,,...,4,,,,,,,,14,1
1,Dua Lipa,Training Season,35,2,3,,,,,,...,1,,,,,,,,2,
2,Royel Otis,Foam,,3,,11,,,,,...,,,,,,,,,,
3,"KUČKA, Flume",One More Night,,4,,,,19,,,...,,,,,,,,,,
4,"Cosmo's Midnight, Franc Moody",Fantasy (feat. Franc Moody),,5,,10,,,,,...,,,,,,,,,,


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Artist                        84 non-null     object
 1   Track                         84 non-null     object
 2   Top 50 Australia              84 non-null     object
 3   New Music Friday AU & NZ      84 non-null     object
 4   Hot Hits Australia            84 non-null     object
 5   Front Left                    84 non-null     object
 6   A1                            84 non-null     object
 7   Dance Generation              84 non-null     object
 8   Get Popped!                   84 non-null     object
 9   R&B Connect                   84 non-null     object
 10  triple j's New Music Hitlist  84 non-null     object
 11  The Flavour                   84 non-null     object
 12  Fresh Country                 84 non-null     object
 13  New Dance Beats       